<h1 style="font-size:3rem;color:orange;">WordleBot v.0.2 - Feedback-based guess space reduction, with letter frequency</h1>

#### > Log Dec 26, 10.30 p.m. : In this version, we try to improve performance by using letter frequency to choose the best guess from a guess space, rather than picking randomly. In testing performance, since the algorithm is deterministic, we only need to parse through 2300 words as answers, instead of having to randomize a sample

## Preliminaries

In [26]:
import matplotlib.pyplot as plt
import time

## Mathematics

In [29]:
def convert_ternary(t):
    """


    Parameters
    ----------
    t : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    Returns
    -------
    int
        Base 10 representation of pattern.

    """
    return sum([t[i]*3**(4-i) for i in range(5)])

In [30]:
def generate_ternary(n):
    """


    Parameters
    ----------
    n : int
        An integer, indicating length of number.

    Returns
    -------
    res : list
        List of all ternary numbers of length n.

    """
    res = list()
    s = [0] * n
    while True:
        temp = s[:]
        res.append(temp)
        i = n-1
        while i >= 0:
            if s[i] == 2:
                i -= 1
            else:
                break
        if i == -1:
            break
        else:
            temp = s[i]
            s[i] = temp + 1
            for j in range(i+1,n):
                s[j] = 0
    return res

## Wordle mechanics

In [31]:
def get_feedback(guess,answer):
    """
    

    Parameters
    ----------
    guess : str
        Five-letter guess.
    answer : str
        Five-letter correct answer.

    Returns
    -------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    """
    #convert string to list
    temp = list(answer)
    answer = temp
    temp = list(guess)
    guess = temp
    
    #initialize
    feedback = ['']*5
    
    #isolate correctly placed letters
    for i in range(5):
        if guess[i] == answer[i]:
            feedback[i] = 2
            answer[i] = ''
            guess[i] = ''
    
    #isolate wrongly placed letters
    for i in range(5):
        if guess[i] == '': continue
        elif guess[i] in answer:
            feedback[i] = 1
            answer[answer.index(guess[i])] = ''
            guess[i] = ''
        else:
            feedback[i] = 0
    
    return feedback

## WordleBot mechanics

In [32]:
def compute_letter_freq(allowed_words):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains all possible words a.k.a guess space at the beginning.

    Returns
    -------
    letter_freq : dict
        Dictionary contains frequency of each of the 26 letters, computed via allowed_words

    """
    letter_freq = dict()
    for word in allowed_words:
        for char in word:
            letter_freq[char] = letter_freq.get(char,0) + 1
    return letter_freq

In [ ]:
def letter_freq_based_score(word,letter_freq):
    return sum([letter_freq[char] for char in set(word)])

In [33]:
def letter_freq_based_choice(still_valid_words,letter_freq):
    """
    

    Parameters
    ----------
    still_valid_words : list
        Contains words still possible as an answer.
    letter_freq : dict
        Dictionary contains frequency of each of the 26 letters, computed via allowed_words (original still_valid_words)

    Returns
    -------
    best_choice : str
        Word that is deemed the best choice, via summing up frequencies of different letters.

    """
    (best_choice,score) = (None,None)
    for word in still_valid_words:
        score = sum([letter_freq[char] for char in set(word)])
        if best_choice_score == None or score > best_choice_score:
            (best_choice,best_choice_score) = (word,score)
    return best_choice

In [ ]:
def get_ranker(allowed_words,letter_freq):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    ranker : list
        Contains ~13000 tuples, whose first element is a guess, and
        second element is the entropy of that guess.

    """
    ranker = list()
    for word in allowed_words:
        ranker.append((word,letter_freq_based_score(word,letter_freq)))
    ranker.sort(key = lambda t: t[1], reverse = True)
    
    return ranker

In [ ]:
def display_ranker(allowed_words,letter_freq):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    None.
    Prints the ranker.

    """
    ranker = get_ranker(allowed_words,letter_freq)
    print('{0:<10}{1:<10}'.format('Word','Score'))
    for (word,entropy) in ranker[:10]: #print only top ten words with highest entropy
        print('{0:<10}{1:<10}'.format(word,entropy))

In [34]:
def reduce_allowed_words(still_valid_words,guess,real_feedback,mode={0,1,2}):
    """
    

    Parameters
    ----------
    still_valid_words : list
        Contains words still possible as an answer.
    guess : str
        Five-letter guess.
    real_feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.
    mode: str
        One of the seven filters: {0}, {1}, {2}, {0,1}, {0,2}, {1,2}, {0,1,2}

    Returns
    -------
    updated_allowed_words : list
        Updates allowed_words by retaining only words fitting the actual feedback.

    """
    n = 0
    accepted_feedback = real_feedback[:]
    
    #letter_feedback works like a mould, designated colors are retained, other positions can be changed arbitrary
    for i, letter_feedback in enumerate(real_feedback):
        if letter_feedback not in mode:
            n += 1
            accepted_feedback[i] = None
    
    accepted_feedbacks_enumerated = list()
    
    #generate_ternary(n) lists all combinations available to fill in the arbitrary positions
    for insertion in generate_ternary(n):
        temp = accepted_feedback[:]
        for i, letter_feedback in enumerate(temp):
            if letter_feedback == None:
                temp[i] = insertion.pop(0) #traverse left to right, popping a combination to fill in the mould
        accepted_feedbacks_enumerated.append(convert_ternary(temp))
    
    updated_allowed_words = list()
    for word in still_valid_words:
        feedback_enumerated = convert_ternary(get_feedback(guess,word))
        if feedback_enumerated in accepted_feedbacks_enumerated:
            updated_allowed_words.append(word)
    
    return updated_allowed_words

## Interactive interface

In [ ]:
def wordlebot_interface(allowed_words,letter_freq,mode={0,1,2}):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    None.
    Prints the interactive program for user to play Wordle and input real feedback.

    """
    win = False
    valid_words = allowed_words
    attempt_number = 0
    while not win:
        print("Guess #" + str(attempt_number+1))
        
        display_ranker(valid_words,letter_freq)
        
        guess = input('> Enter your guess: ')
        real_feedback = list(map(int,input('>> Enter the feedback: ').split(' ')))
        
        
        if check_win(real_feedback) == True:
            print(">>> Complete!")
            break
        
        temp = reduce_allowed_words(valid_words,guess,real_feedback)
        valid_words = temp
        print(">>> Number of remaining words in the guess space: " + str(len(valid_words)) + "\n")
        
        attempt_number += 1

## Performance testing

In [35]:
def check_win(feedback):
    """
    

    Parameters
    ----------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2.

    Returns
    -------
    win : bool
        Becomes True when feedback is a list of 05 2's.

    """
    win = True
    for i in range(5):
        if feedback[i] != 2: 
            win = False
            break
    return win

In [36]:
def wordlebot_play(allowed_words,answer,letter_freq,mode={0,1,2}):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains ~13000 allowed guesses.
    answer : str
        Five-letter actual answer.
    mode : set
        A set indicating one of the seven modes: {0}, {1}, {2}, {0,1}, {1,2}, {0,2}, {0,1,2}

    Returns
    -------
    guess_count : int
        Number of guesses needed to reach the actual answer.

    """
    import random
    still_valid_words = allowed_words
    guess_count = 0
    
    while True:
        
        win = len(still_valid_words) == 1
        
        guess = letter_freq_based_choice(still_valid_words,letter_freq)
        
        guess_count += 1
        
        real_feedback = get_feedback(guess,answer)
        
        if check_win(real_feedback):
            break
            
        temp = reduce_allowed_words(still_valid_words,guess,real_feedback,mode)
        still_valid_words = temp
    
    return guess_count

In [45]:
def test_for_performance(allowed_words,possible_answers,n="all",mode={0,1,2}):
    """
    

    Parameters
    ----------
    possible_answers : list
        Contains ~2300 human-curated possible answers.
    n : int
        Default to "all" - if n is not given, test on all possible answers.
        If n is given, test on first n answers of all possible answers.
    mode : set
        A set indicating one of the seven modes: {0}, {1}, {2}, {0,1}, {1,2}, {0,2}, {0,1,2}

    Returns
    -------
    None.
    Prints bar plot showing frequency of number of guesses needed.

    """
    #initialize
    total = len(possible_answers)
    performance_count = dict()
    
    #gameplay for ~2300 words in POSSIBLE_ANSWERS
    count = 1
    if n == "all":
        for word in possible_answers:
            guess_count = wordlebot_play(allowed_words,word,mode)

            performance_count[guess_count] = performance_count.get(guess_count,0) + 1
            print("Word " + str(count) + "/" + str(total) + ": " + word + " - Guesses taken: " + str(guess_count))
            count += 1
            
    else:
        for word in possible_answers[:n]:
            guess_count = wordlebot_play(allowed_words,word,mode)

            performance_count[guess_count] = performance_count.get(guess_count,0) + 1
            print("Word " + str(count) + "/" + str(n) + ": " + word + " - Guesses taken: " + str(guess_count))
            count += 1
    
    #visualize
    x = list(range(1,max(performance_count.keys())+1))
    y = [performance_count.get(i,0) for i in x]
    plt.bar(x,y,color='royalblue',alpha=0.7)
    plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.7)
    plt.title('WordleBot - Test performance')
    plt.xlabel('Number of guesses needed')
    plt.ylabel('Frequency')
    plt.show()
        
    #average number of guesses needed
    sum = 0
    win_count = 0
    for (guess_count,frequency) in performance_count.items():
        if guess_count <= 6:
            win_count += frequency
        sum += guess_count * frequency
    if n == "all":
        average = sum/len(possible_answers)
        win_rate = win_count/len(possible_answers)
    else:
        average = sum/n
        win_rate = win_count/n
    print("> Average number of guesses needed:",average)
    print("> Win rate:",win_rate) 

    #detailed distribution
    print("> Detailed distribution of number of guesses needed:")
    for guess_count in sorted(performance_count):
        print("- " + str(guess_count) + " guess(es): " + str(performance_count[guess_count]))

In [ ]:
def main():
    f = open('allowed_words.txt','r')
    ALLOWED_WORDS = list()
    for line in f:
        line = line.rstrip()
        ALLOWED_WORDS.append(line)
    f.close()

    f = open('possible_answers.txt','r')
    POSSIBLE_ANSWERS = list()
    for line in f:
        line = line.rstrip()
        POSSIBLE_ANSWERS.append(line)
    f.close()

    LETTER_FREQ = compute_letter_freq(ALLOWED_WORDS)

    wordlebot_interface(ALLOWED_WORDS,LETTER_FREQ)  

In [ ]:
if __name__ == "__main__":
    main()